<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/license_plate_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install

In [ ]:
!pip install ultralytics
!pip install easyocr
!pip install pytube opencv-python-headless

In [3]:
from ultralytics import YOLO
from IPython.display import display, Image

#Test

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Install YOLOv8, EasyOCR, and OpenCV
# !pip install ultralytics
# !pip install easyocr
# !pip install opencv-python-headless

# Step 2: Import Required Libraries
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import easyocr
import numpy as np

# Step 3: Load the YOLOv8 Model with Custom Weights
model = YOLO('/content/drive/MyDrive/license plate/model.pt')  # Replace with the path to your weights file

# Step 4: Load the Input Video
input_video_path = '/content/drive/MyDrive/license plate/test.mp4'  # Replace with the path to your input video
cap = cv2.VideoCapture(input_video_path)

# Step 5: Initialize EasyOCR Reader for Thai Language
reader = easyocr.Reader(['th'])  # Specify 'th' for Thai language

# Step 6: Process Each Frame
output_video_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Step 7: Perform Inference
    results = model.predict(image_rgb)

    # Step 8: Process Each Detection
    output_frame = image_rgb.copy()

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Access the bounding box coordinates
            class_id = int(box.cls)  # Access the class ID

            # Draw bounding box
            cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Convert the output_frame back to BGR for saving
    output_frame_bgr = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    out.write(output_frame_bgr)

    # # Optionally, display the frame
    # cv2.imshow('Output Video', output_frame_bgr)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Step 9: Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()

##Apply Tracking

In [ ]:
# Step 1: Install YOLOv8, EasyOCR, and OpenCV
# !pip install ultralytics
# !pip install easyocr
# !pip install opencv-python-headless

# Step 2: Import Required Libraries
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import easyocr
import numpy as np

# Step 3: Load the YOLOv8 Model with Custom Weights
model = YOLO('/content/drive/MyDrive/license plate/model.pt')  # Replace with the path to your weights file

# Step 4: Load the Input Video
input_video_path = '/content/drive/MyDrive/license plate/test.mp4'  # Replace with the path to your input video
cap = cv2.VideoCapture(input_video_path)

# Step 5: Initialize EasyOCR Reader for Thai Language
reader = easyocr.Reader(['th'])  # Specify 'th' for Thai language

# Step 6: Initialize Video Writer
output_video_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Step 7: Initialize Tracker Dictionary
trackers = {}

# Step 8: Process Each Frame
frame_id = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Step 9: Perform Inference on the first frame and every 10 frames
    if frame_id % 10 == 0:
        results = model.predict(image_rgb)
        new_trackers = []

        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Access the bounding box coordinates

                # Initialize a new tracker for each detected object
                tracker = cv2.TrackerCSRT_create()
                tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                new_trackers.append(tracker)

        # Update the trackers dictionary with new trackers
        trackers = {i: tracker for i, tracker in enumerate(new_trackers)}

    # Step 10: Update trackers and draw bounding boxes
    output_frame = image_rgb.copy()
    for tracker in trackers.values():
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = map(int, bbox)
            cv2.rectangle(output_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Convert the output_frame back to BGR for saving
    output_frame_bgr = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    out.write(output_frame_bgr)

    frame_id += 1

    # Optionally, display the frame
    # cv2.imshow('Output Video', output_frame_bgr)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Step 11: Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()

## Crop

In [ ]:
# Step 1: Install YOLOv8, EasyOCR, and OpenCV
# !pip install ultralytics
# !pip install easyocr
# !pip install opencv-python-headless

# Step 2: Import Required Libraries
import cv2
from ultralytics import YOLO
import easyocr
import os

# Step 3: Load the YOLOv8 Model with Custom Weights
model = YOLO('/content/drive/MyDrive/license plate/model.pt')  # Replace with the path to your weights file

# Step 4: Load the Input Video
input_video_path = '/content/drive/MyDrive/license plate/test2.mp4'  # Replace with the path to your input video
cap = cv2.VideoCapture(input_video_path)

# Step 5: Initialize EasyOCR Reader for Thai Language
reader = easyocr.Reader(['th'])  # Specify 'th' for Thai language

# Step 6: Initialize Video Writer
output_video_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Create directory to save detected objects
save_dir = '/content/detected_objects'
os.makedirs(save_dir, exist_ok=True)

# Dictionary to track if a class has been saved
saved_classes = {}

# Step 7: Initialize Tracker Dictionary
trackers = {}

# Step 8: Process Each Frame
frame_id = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Step 9: Perform Inference on the first frame and every 10 frames
    if frame_id % 20 == 0:
        results = model.predict(image_rgb)
        new_trackers = []

        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Access the bounding box coordinates

                # Initialize a new tracker for each detected object
                tracker = cv2.TrackerKCF_create()
                tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                new_trackers.append(tracker)

                # Save the detected object as an image if it hasn't been saved already
                class_id = int(box.cls[0])
                class_name = result.names[class_id]
                if class_name not in saved_classes:
                    cropped_img = frame[y1:y2, x1:x2]
                    save_path = os.path.join(save_dir, f'{class_name}.jpg')
                    cv2.imwrite(save_path, cropped_img)
                    saved_classes[class_name] = True

        # Update the trackers dictionary with new trackers
        trackers = {i: tracker for i, tracker in enumerate(new_trackers)}

    # Step 10: Update trackers and draw bounding boxes
    output_frame = image_rgb.copy()
    for tracker in trackers.values():
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = map(int, bbox)
            cv2.rectangle(output_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Convert the output_frame back to BGR for saving
    output_frame_bgr = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    out.write(output_frame_bgr)

    frame_id += 1

# Step 11: Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()

## OCR

In [ ]:
!git clone https://github.com/nannthd/EasyOCR_license_plate

In [32]:
%cd /content/EasyOCR_license_plate

/content/EasyOCR_license_plate


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install Levenshtein

In [ ]:
# Step 1: Install EasyOCR and OpenCV if not installed
# !pip install easyocr
# !pip install opencv-python-headless

# Step 2: Import Required Libraries
import cv2
import easyocr
import os
import matplotlib.pyplot as plt
import Levenshtein as lev

# Step 3: Initialize EasyOCR Reader for Thai Language
reader = easyocr.Reader(['th'])  # Specify 'th' for Thai language

# Step 4: Specify the Directory Containing Cropped Images
cropped_images_dir = '/content/detected_objects'

# Step 5: Specify the Dictionary Path
th_dict_path = '/content/EasyOCR_license_plate/easyocr/dict/th.txt'

# Load words from the dictionary
with open(th_dict_path, 'r', encoding='utf-8') as f:
    thai_words = [word.strip() for word in f.readlines()]

# Step 6: Initialize a Dictionary to Store OCR Results
detected_texts = {'text_number': [], 'text_province': []}

# Step 7: Perform OCR on Each Cropped Image
for image_name in os.listdir(cropped_images_dir):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(cropped_images_dir, image_name)
        class_name = os.path.splitext(image_name)[0]

        # Load the image
        cropped_image = cv2.imread(image_path)

        # Apply EasyOCR to the image
        ocr_result = reader.readtext(cropped_image, detail=0)

        selected_texts = []
        for text in ocr_result:
            closest_word = text

            # Find the closest word from the dictionary using Levenshtein distance
            if 'province' in class_name.lower():  # Check only for Province
                min_distance = float('inf')
                best_match = text

                for th_word in thai_words:
                    dist = lev.distance(text, th_word)
                    if dist < min_distance:
                        min_distance = dist
                        best_match = th_word

                selected_texts.append(best_match)
            else:
                selected_texts.append(text)

        ocr_text = ' '.join(selected_texts)
        print(f"OCR Result for {class_name}: {ocr_text}")

        # Categorize the detected text
        if 'number' in class_name.lower():
            detected_texts['text_number'].extend(selected_texts)
        elif 'province' in class_name.lower():
            detected_texts['text_province'].extend(selected_texts)

# Step 8: Display the Images
number_image_path = '/content/detected_objects/number.jpg'
province_image_path = '/content/detected_objects/province.jpg'

number_image = cv2.imread(number_image_path)
province_image = cv2.imread(province_image_path)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(number_image, cv2.COLOR_BGR2RGB))
plt.title('Number Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(province_image, cv2.COLOR_BGR2RGB))
plt.title('Province Image')
plt.axis('off')

plt.tight_layout()
plt.show()